In [2]:
import pandas as pd
import numpy as np
import string
import pickle

from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

In [3]:
train = pd.read_csv('trainMaster.csv')
final = pd.read_csv('dfTestDataLabelFinal.csv')

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 15 columns):
Unnamed: 0       159571 non-null int64
id               159571 non-null object
comment_text     159571 non-null object
toxic            159571 non-null int64
severe_toxic     159571 non-null int64
obscene          159571 non-null int64
threat           159571 non-null int64
insult           159571 non-null int64
identity_hate    159571 non-null int64
toxic_level      159571 non-null int64
clean            159571 non-null int64
length           159571 non-null int64
word_count       159571 non-null int64
polarity         159571 non-null float64
subjectivity     159571 non-null float64
dtypes: float64(2), int64(11), object(2)
memory usage: 18.3+ MB


In [6]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63978 entries, 0 to 63977
Data columns (total 14 columns):
id               63978 non-null object
comment_text     63978 non-null object
toxic            63978 non-null int64
severe_toxic     63978 non-null int64
obscene          63978 non-null int64
threat           63978 non-null int64
insult           63978 non-null int64
identity_hate    63978 non-null int64
toxic_level      63978 non-null int64
clean            63978 non-null int64
length           63978 non-null int64
word_count       63978 non-null int64
polarity         63978 non-null float64
subjectivity     63978 non-null float64
dtypes: float64(2), int64(10), object(2)
memory usage: 6.8+ MB


In [3]:
def preprocess(ReviewText):
    ReviewText = ReviewText.str.replace("(<br/>)", "")
    ReviewText = ReviewText.str.replace('(<a).*(>).*(</a>)', '')
    ReviewText = ReviewText.str.replace('(&amp)', '')
    ReviewText = ReviewText.str.replace('(&gt)', '')
    ReviewText = ReviewText.str.replace('(&lt)', '')
    ReviewText = ReviewText.str.replace('(\xa0)', ' ')
    ReviewText = ReviewText.str.replace('(\n)', ' ') 
    ReviewText = ReviewText.str.replace('==', ' ') 
    return ReviewText

train['comment_text'] = preprocess(train['comment_text'])
final['comment_text'] = preprocess(final['comment_text'])

# Target

In [4]:
x = train['comment_text']

categories = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']

# Train Test Split

In [5]:
# from sklearn.model_selection import train_test_split

In [6]:
# train, test = train_test_split(train, random_state=42, test_size=0.30, shuffle=True)

In [7]:
# print('OwO')

OwO


# K Fold

In [8]:
# from sklearn.model_selection import KFold

In [9]:
# K = 5
# kf = KFold(n_splits=K, shuffle=True, random_state = 42)

In [10]:
# dFold = {}
# for i in range(1,8):
#     dFold['Hasilfold{}'.format(i)] = kf.split(x,datay[i-1])

# Model

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, matthews_corrcoef, f1_score, log_loss

# Analyzer for Pipeline

In [12]:
stop = []
for words in stopwords.words('english'):
    s = [char for char in words if char not in string.punctuation]
    stop.append(''.join(s))
stop.extend(['may','also','across','among','beside','however','yet','within'])

# stemmer = SnowballStemmer("english")
# def stemming(sentence):
#     stemSentence = ""
#     for word in sentence.split():
#         stem = stemmer.stem(word)
#         stemSentence += stem
#         stemSentence += " "
#     stemSentence = stemSentence.strip()
#     return stemSentence

def process_normal(text):
    return [word for word in text.split() if word.lower() not in stop]

def process_nocase(text):
    return [word.lower() for word in text.split() if word.lower() not in stop]

def process_nopunc(text):
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in stop]

def process_nocasepunc(text):
    nocasepunc = [char for char in text if char not in string.punctuation]
    nocasepunc = ''.join(nocasepunc)
    return [word.lower() for word in nocasepunc.split() if word.lower() not in stop]

 # Vectorizer

In [13]:
vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,3), stop_words='english', max_features=30000)
# vectorizer = TfidfVectorizer(, analyzer='word', ngram_range=(1,3), norm='l2')
#     norm = ('l1','l2','none')
#     max_features  = (10000,50000,100000,150000)

# Pipeline

In [14]:
LogReg_pipeline = Pipeline([
#             ('vectorizer', TfidfVectorizer(analyzer='word', ngram_range=(1,3), max_features=30000)),
            ('classifier', LogisticRegression()),
        ])

def pipeline_model_training(category, dataTrain, targetTrain):
    # Training logistic regression model on train data
    return LogReg_pipeline.fit(dataTrain, targetTrain)

def pipeline_predict(category, dictPred, dictPredProba, dictHasil, dataTest, targetTest):
    # calculating test accuracy
    prediction = LogReg_pipeline.predict(dataTest)
    dictPredProba['{}'.format(category)] = LogReg_pipeline.predict_proba(dataTest)
    dictPred[category] = prediction 
    dictHasil['{}_accuracy'.format(category)] = accuracy_score(targetTest, prediction)
    dictHasil['{}_f1score'.format(category)] = f1_score(targetTest, prediction, average = 'macro')
    dictHasil['{}_logloss'.format(category)] = log_loss(targetTest, dictPredProba[category])
    dictHasil['{}_rocauc'.format(category)] = roc_auc_score(targetTest, dictPredProba[category][:,1])
    dictHasil['{}_matthews_corrcoef'.format(category)] = matthews_corrcoef(targetTest, prediction)
    dictHasil['{}_classification_report'.format(category)] = classification_report(targetTest, prediction)
    dictHasil['{}_confusion_matrix'.format(category)] = classification_report(targetTest, prediction)
    

In [15]:
print('OwO')

OwO


# Gridsearch

In [16]:
# for category in categories:
#     norm = ('l1','l2','none')
#     max_features  = (10000,50000,100000,150000)


#     param_grid = {'norm': norm, 'max_features':max_features, }

#     gs = GridSearchCV(LogReg_pipeline.steps[0][1],param_grid,scoring='accuracy')


#     gs = gs.fit(x_train, y_train[category])
    
#     print(category)
#     print(gs.best_score_)
#     print(gs.best_params_)

# Predict Train

In [17]:
vectorizer.fit(train['comment_text'])

x_train = vectorizer.transform(train['comment_text'])
y_train = train.drop(labels = ['id','comment_text'], axis=1)
x_test = vectorizer.transform(test['comment_text'])
y_test = test.drop(labels = ['id','comment_text'], axis=1)

In [18]:
dPredict_train = {}
dPredictProba_train = {}
dResult_train = {}

for category in categories:
    pipeline_model_training(category, x_train, y_train[category])
    pipeline_predict(category, dPredict_train, dPredictProba_train, dResult_train, x_train, y_train[category])

C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this w

## Report and Analysis

In [19]:
def bikin_report(hasil, namadf, namacol):
    listItem = []

    for category in categories:
        listItem.append([category,
                        hasil['{}_accuracy'.format(category)],
                        hasil['{}_f1score'.format(category)],
                        hasil['{}_logloss'.format(category)],
                        hasil['{}_rocauc'.format(category)],
                        hasil['{}_matthews_corrcoef'.format(category)],
                        ])

    namadf = pd.DataFrame(columns=['Category','{} Acc'.format(namacol), '{} F1 Score'.format(namacol),'{} Log Loss'.format(namacol),'{} ROC AUC'.format(namacol),'{} Matthew Corr Coef'.format(namacol),],
                         data=listItem)
    return namadf

In [20]:
dfReportTrain = bikin_report(dResult_train, 'dfReportTrain', 'Train')
dfReportTrain

,Category,Train Acc,Train F1 Score,Train Log Loss,Train ROC AUC,Train Matthew Corr Coef
0,toxic,0.959561,0.861242,0.107165,0.984151,0.743385
1,severe_toxic,0.990752,0.657198,0.025001,0.991896,0.356164
2,obscene,0.979158,0.877397,0.057750,0.992887,0.769900
3,threat,0.997162,0.583279,0.010293,0.995216,0.276518
4,insult,0.973124,0.823514,0.071457,0.987037,0.668873
5,identity_hate,0.992417,0.643701,0.023826,0.988702,0.372215


In [21]:
dfReportTrain.describe()

,Train Acc,Train F1 Score,Train Log Loss,Train ROC AUC,Train Matthew Corr Coef
count,6.000000,6.000000,6.000000,6.000000,6.000000
mean,0.982029,0.741055,0.049249,0.989981,0.531176
std,0.014180,0.127465,0.036528,0.004094,0.219878
min,0.959561,0.583279,0.010293,0.984151,0.276518
25%,0.974633,0.647075,0.024120,0.987453,0.360177
50%,0.984955,0.740356,0.041376,0.990299,0.520544
75%,0.992001,0.851810,0.068031,0.992639,0.724757
max,0.997162,0.877397,0.107165,0.995216,0.769900


In [22]:
def bikin_hasil_pred(hasil, namadf):
    listItem = []
    
    for i in range(len(hasil['toxic'])):
        listItem.append([hasil['toxic'][i],
                        hasil['severe_toxic'][i],
                        hasil['obscene'][i],
                        hasil['threat'][i],
                        hasil['threat'][i],
                        hasil['identity_hate'][i],])
        
        
        if((hasil['toxic'][i] + hasil['severe_toxic'][i] + hasil['obscene'][i] + hasil['threat'][i] + hasil['threat'][i] + hasil['identity_hate'][i]) == 0):
            listItem[i].append(1)
        else:
            listItem[i].append(0)
    

    namadf = pd.DataFrame(columns=['ToxicPred', 'SToxicPred','ObscenePred','ThreatPred','InsultPred','IdHatePred', 'cleanPred'],
                         data=listItem)
    return namadf

In [23]:
dfResultTrain = bikin_hasil_pred(dPredict_train, 'dfResultTrain')
dfResultTrain.head(8)

,ToxicPred,SToxicPred,ObscenePred,ThreatPred,InsultPred,IdHatePred,cleanPred
0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1
5,0,0,0,0,0,0,1
6,0,0,0,0,0,0,1
7,1,0,1,0,0,0,0


In [24]:
def combine_predict_with_keylabel(namadf, pred, key):
    namadf = pd.DataFrame
    namadf = pd.concat([pred, key[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate', 'clean']].reset_index(drop=True)], axis=1)
    return namadf

In [25]:
dfFinalTrain = combine_predict_with_keylabel('dfFinalTrain', dfResultTrain, y_train)

In [26]:
dfFinalTrain.head()

,ToxicPred,SToxicPred,ObscenePred,ThreatPred,InsultPred,IdHatePred,cleanPred,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean
0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1,0,0,0,0,0,0,1


In [27]:
def filter_dataframe(dfResult):
    actCancer = dfResult[dfResult['clean']==0]
    actClean = dfResult[dfResult['clean']==1]
    
    predCancer = dfResult[((dfResult['ToxicPred'] == 1) | (dfResult['SToxicPred'] == 1) | (dfResult['ObscenePred'] == 1)
             | (dfResult['ThreatPred'] == 1) | (dfResult['InsultPred'] == 1) | (dfResult['IdHatePred'] == 1))]
    
    predCancerActCancer = dfResult[((dfResult['ToxicPred'] == dfResult['toxic']) & (dfResult['SToxicPred'] == dfResult['severe_toxic']) & (dfResult['ObscenePred'] == dfResult['obscene'])
             & (dfResult['ThreatPred'] == dfResult['threat']) & (dfResult['InsultPred'] == dfResult['insult']) & (dfResult['IdHatePred'] == dfResult['identity_hate'])) 
                                   & (dfResult['clean']==0)]
    
    predClean = dfResult[((dfResult['ToxicPred'] == 0) & (dfResult['SToxicPred'] == 0) & (dfResult['ObscenePred'] == 0)
             & (dfResult['ThreatPred'] == 0) & (dfResult['InsultPred'] == 0) & (dfResult['IdHatePred'] == 0))]
    
    predCleanActClean = dfResult[((dfResult['ToxicPred'] == dfResult['toxic']) & (dfResult['SToxicPred'] == dfResult['severe_toxic']) & (dfResult['ObscenePred'] == dfResult['obscene'])
             & (dfResult['ThreatPred'] == dfResult['threat']) & (dfResult['InsultPred'] == dfResult['insult']) & (dfResult['IdHatePred'] == dfResult['identity_hate'])) & (dfResult['clean']==1)]

    predTrueActTrue = dfResult[(dfResult['ToxicPred'] == dfResult['toxic']) & (dfResult['SToxicPred'] == dfResult['severe_toxic']) & (dfResult['ObscenePred'] == dfResult['obscene']) & (dfResult['ThreatPred'] == dfResult['threat']) & (dfResult['InsultPred'] == dfResult['insult']) & (dfResult['IdHatePred'] == dfResult['identity_hate'])]

    return actCancer, actClean, predCancer, predCancerActCancer, predClean, predCleanActClean, predTrueActTrue

In [28]:
actCancer_train, actClean_train, predCancer_train, predCancerActCancer_train, predClean_train, predCleanActClean_train, predTrueActTrue_train = filter_dataframe(dfFinalTrain)

In [29]:
def print_filter(actCancer, actClean, predCancer, predCancerActCancer, predClean, predCleanActClean,  predTrueActTrue):
    print('Cancer = Data yang mengandung salah satu dari toxic, severe toxic, obscene, threat, insult, atau identity hate')
    print('\n')
    print('Total Data di Dataframe = {}'.format(len(actCancer)+len(actClean)))
    print('Banyak data Cancer = {}'.format(len(actCancer)))
    print('Banyak data Clean = {}'.format(len(actClean)))
    print('\n')
    print('Banyak Prediction Cancer yang benar = {}'.format(len(predCancerActCancer)))
    print('Percentage dari Prediction Cancer yang benar = {}%'.format(len(predCancerActCancer)/len(actCancer)*100))
    print('\n')
    print('Banyak Prediction Clean yang benar = {}'.format(len(predCleanActClean)))
    print('Percentage dari Prediction Clean yang benar = {}%'.format(len(predCleanActClean)/len(actClean)*100))
    print('\n')
    print('Banyak Prediction yang benar = {}'.format(len(predTrueActTrue)))
    print('Percentage dari Prediction yang benar = {}%'.format(len(predTrueActTrue)/(len(actCancer)+len(actClean))*100))

In [30]:
print_filter(actCancer_train, actClean_train, predCancer_train, predCancerActCancer_train, predClean_train, predCleanActClean_train, predTrueActTrue_train)

Cancer = Data yang mengandung salah satu dari toxic, severe toxic, obscene, threat, insult, atau identity hate


Total Data di Dataframe = 111699
Banyak data Cancer = 11358
Banyak data Clean = 100341


Banyak Prediction Cancer yang benar = 1769
Percentage dari Prediction Cancer yang benar = 15.57492516288079%


Banyak Prediction Clean yang benar = 100073
Percentage dari Prediction Clean yang benar = 99.73291077425978%


Banyak Prediction yang benar = 101842
Percentage dari Prediction yang benar = 91.17539100618627%


In [31]:
actCancer = dfFinalTrain[dfFinalTrain['clean']==0]
actClean = dfFinalTrain[dfFinalTrain['clean']==1]
predCancer = dfFinalTrain[dfFinalTrain['cleanPred']==0]
predClean = dfFinalTrain[dfFinalTrain['cleanPred']==1]

len(actCancer), len(actClean), len(predCancer), len(predClean), 

(11358, 100341, 6972, 104727)

In [32]:
actCancer2 = dfFinalTrain[(dfFinalTrain['clean']==0) & ((dfFinalTrain['ToxicPred'] == dfFinalTrain['toxic']) & (dfFinalTrain['SToxicPred'] == dfFinalTrain['severe_toxic']) & (dfFinalTrain['ObscenePred'] == dfFinalTrain['obscene']) & (dfFinalTrain['ThreatPred'] == dfFinalTrain['threat']) & (dfFinalTrain['InsultPred'] == dfFinalTrain['insult']) & (dfFinalTrain['IdHatePred'] == dfFinalTrain['identity_hate']))]
actClean2 = dfFinalTrain[(dfFinalTrain['clean']==1) & ((dfFinalTrain['ToxicPred'] == dfFinalTrain['toxic']) & (dfFinalTrain['SToxicPred'] == dfFinalTrain['severe_toxic']) & (dfFinalTrain['ObscenePred'] == dfFinalTrain['obscene']) & (dfFinalTrain['ThreatPred'] == dfFinalTrain['threat']) & (dfFinalTrain['InsultPred'] == dfFinalTrain['insult']) & (dfFinalTrain['IdHatePred'] == dfFinalTrain['identity_hate']))]
predCancer2 = dfFinalTrain[(dfFinalTrain['cleanPred']==0) & ((dfFinalTrain['ToxicPred'] == dfFinalTrain['toxic']) & (dfFinalTrain['SToxicPred'] == dfFinalTrain['severe_toxic']) & (dfFinalTrain['ObscenePred'] == dfFinalTrain['obscene']) & (dfFinalTrain['ThreatPred'] == dfFinalTrain['threat']) & (dfFinalTrain['InsultPred'] == dfFinalTrain['insult']) & (dfFinalTrain['IdHatePred'] == dfFinalTrain['identity_hate']))]
predClean2 = dfFinalTrain[(dfFinalTrain['cleanPred']==1) & ((dfFinalTrain['ToxicPred'] == dfFinalTrain['toxic']) & (dfFinalTrain['SToxicPred'] == dfFinalTrain['severe_toxic']) & (dfFinalTrain['ObscenePred'] == dfFinalTrain['obscene']) & (dfFinalTrain['ThreatPred'] == dfFinalTrain['threat']) & (dfFinalTrain['InsultPred'] == dfFinalTrain['insult']) & (dfFinalTrain['IdHatePred'] == dfFinalTrain['identity_hate']))]

len(actCancer2), len(actClean2), len(predCancer2), len(predClean2), 

(1769, 100073, 1769, 100073)

# 1684 9674 = 11358
# 222 100119 = 100 341
# 6790 104909 101803

# Predict Test

In [34]:
dPredict_test = {}
dPredictProba_test = {}
dResult_test = {}

for category in categories:
    pipeline_predict(category, dPredict_test, dPredictProba_test, dResult_test, x_test, y_test[category])

## Report and Analysis

In [35]:
dfReportTest = bikin_report(dResult_test, 'dfReportTest', 'Test')
dfReportTest

,Category,Test Acc,Test F1 Score,Test Log Loss,Test ROC AUC,Test Matthew Corr Coef
0,toxic,0.905853,0.491986,0.395822,0.898878,0.121359
1,severe_toxic,0.989576,0.557343,0.036930,0.965534,0.168217
2,obscene,0.947673,0.511576,0.202670,0.929993,0.139435
3,threat,0.995634,0.517339,0.019694,0.934112,0.036000
4,insult,0.951245,0.515019,0.189308,0.928008,0.149289
5,identity_hate,0.991874,0.618815,0.028702,0.971729,0.329301


In [36]:
dfReportTest.describe()

,Test Acc,Test F1 Score,Test Log Loss,Test ROC AUC,Test Matthew Corr Coef
count,6.000000,6.000000,6.000000,6.000000,6.000000
mean,0.963643,0.535346,0.145521,0.938042,0.157267
std,0.035331,0.046116,0.147719,0.026853,0.096013
min,0.905853,0.491986,0.019694,0.898878,0.036000
25%,0.948566,0.512437,0.030759,0.928504,0.125878
50%,0.970411,0.516179,0.113119,0.932053,0.144362
75%,0.991300,0.547342,0.199330,0.957678,0.163485
max,0.995634,0.618815,0.395822,0.971729,0.329301


In [37]:
dfResultTest = bikin_hasil_pred(dPredict_test, 'dfResultTest')
dfResultTest.head(8)

,ToxicPred,SToxicPred,ObscenePred,ThreatPred,InsultPred,IdHatePred,cleanPred
0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1
5,0,0,0,0,0,0,1
6,0,0,0,0,0,0,1
7,0,0,0,0,0,0,1


In [38]:
dfFinalTest = combine_predict_with_keylabel('dfFinalTest', dfResultTest, y_test)

In [39]:
dfFinalTest.head()

,ToxicPred,SToxicPred,ObscenePred,ThreatPred,InsultPred,IdHatePred,cleanPred,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean
0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1,0,0,0,0,0,0,1


In [40]:
actCancer_test, actClean_test, predCancer_test, predCancerActCancer_test, predClean_test, predCleanActClean_test, predTrueActTrue_test = filter_dataframe(dfFinalTest)

In [41]:
print_filter(actCancer_test, actClean_test, predCancer_test, predCancerActCancer_test, predClean_test, predCleanActClean_test, predTrueActTrue_test)

Cancer = Data yang mengandung salah satu dari toxic, severe toxic, obscene, threat, insult, atau identity hate


Total Data di Dataframe = 47872
Banyak data Cancer = 4867
Banyak data Clean = 43005


Banyak Prediction Cancer yang benar = 0
Percentage dari Prediction Cancer yang benar = 0.0%


Banyak Prediction Clean yang benar = 43002
Percentage dari Prediction Clean yang benar = 99.99302406696896%


Banyak Prediction yang benar = 43002
Percentage dari Prediction yang benar = 89.82703877005348%


# Predict Final

In [42]:
x_final = vectorizer.transform(final['comment_text'])
y_final = final.drop(labels = ['id','comment_text'], axis=1)

In [43]:
dPredict_final = {}
dPredictProba_final = {}
dResult_final = {}

for category in categories:
    pipeline_predict(category, dPredict_final, dPredictProba_final, dResult_final, x_final, y_final[category])

## Report and Analysis

In [44]:
dfReportFinal = bikin_report(dResult_final, 'dfReportFinal', 'Final')
dfReportFinal

,Category,Final Acc,Final F1 Score,Final Log Loss,Final ROC AUC,Final Matthew Corr Coef
0,toxic,0.906874,0.502056,0.372727,0.905306,0.140286
1,severe_toxic,0.992794,0.593094,0.025303,0.975413,0.194940
2,obscene,0.943402,0.520341,0.213665,0.920549,0.148585
3,threat,0.993795,0.517814,0.024234,0.942090,0.035638
4,insult,0.947748,0.525960,0.197826,0.915951,0.163559
5,identity_hate,0.990028,0.648474,0.032372,0.975166,0.360535


In [45]:
dfReportFinal.describe()

,Final Acc,Final F1 Score,Final Log Loss,Final ROC AUC,Final Matthew Corr Coef
count,6.000000,6.000000,6.000000,6.000000,6.000000
mean,0.962440,0.551290,0.144355,0.939079,0.173924
std,0.035585,0.057160,0.142098,0.030497,0.106084
min,0.906874,0.502056,0.024234,0.905306,0.035638
25%,0.944489,0.518446,0.027070,0.917100,0.142361
50%,0.968888,0.523151,0.115099,0.931320,0.156072
75%,0.992103,0.576311,0.209705,0.966897,0.187094
max,0.993795,0.648474,0.372727,0.975413,0.360535


In [46]:
dfResultFinal = bikin_hasil_pred(dPredict_final, 'dfResultFinal')
dfResultFinal.head(8)

,ToxicPred,SToxicPred,ObscenePred,ThreatPred,InsultPred,IdHatePred,cleanPred
0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1
5,0,0,0,0,0,0,1
6,0,0,0,0,0,0,1
7,0,0,0,0,0,0,1


In [47]:
dfFinalFinal = combine_predict_with_keylabel('dfFinalFinal', dfResultFinal, final)

In [48]:
dfFinalFinal.head()

,ToxicPred,SToxicPred,ObscenePred,ThreatPred,InsultPred,IdHatePred,cleanPred,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean
0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1,0,0,0,0,0,0,1


In [49]:
actCancer_final, actClean_final, predCancer_final, predCancerActCancer_final, predClean_final, predCleanActClean_final, predTrueActTrue_final = filter_dataframe(dfFinalFinal)

In [50]:
print_filter(actCancer_final, actClean_final, predCancer_final, predCancerActCancer_final, predClean_final, predCleanActClean_final, predTrueActTrue_final)

Cancer = Data yang mengandung salah satu dari toxic, severe toxic, obscene, threat, insult, atau identity hate


Total Data di Dataframe = 63978
Banyak data Cancer = 6243
Banyak data Clean = 57735


Banyak Prediction Cancer yang benar = 4
Percentage dari Prediction Cancer yang benar = 0.06407176037161622%


Banyak Prediction Clean yang benar = 57702
Percentage dari Prediction Clean yang benar = 99.94284229670043%


Banyak Prediction yang benar = 57706
Percentage dari Prediction yang benar = 90.19663009159399%


In [51]:
list(dResult_final.keys())

['toxic_accuracy',
 'toxic_f1score',
 'toxic_logloss',
 'toxic_rocauc',
 'toxic_matthews_corrcoef',
 'toxic_classification_report',
 'toxic_confusion_matrix',
 'severe_toxic_accuracy',
 'severe_toxic_f1score',
 'severe_toxic_logloss',
 'severe_toxic_rocauc',
 'severe_toxic_matthews_corrcoef',
 'severe_toxic_classification_report',
 'severe_toxic_confusion_matrix',
 'obscene_accuracy',
 'obscene_f1score',
 'obscene_logloss',
 'obscene_rocauc',
 'obscene_matthews_corrcoef',
 'obscene_classification_report',
 'obscene_confusion_matrix',
 'threat_accuracy',
 'threat_f1score',
 'threat_logloss',
 'threat_rocauc',
 'threat_matthews_corrcoef',
 'threat_classification_report',
 'threat_confusion_matrix',
 'insult_accuracy',
 'insult_f1score',
 'insult_logloss',
 'insult_rocauc',
 'insult_matthews_corrcoef',
 'insult_classification_report',
 'insult_confusion_matrix',
 'identity_hate_accuracy',
 'identity_hate_f1score',
 'identity_hate_logloss',
 'identity_hate_rocauc',
 'identity_hate_matthews

In [52]:
for category in categories:
    print(str(category))
    print(dResult_final['{}_confusion_matrix'.format(category)])
    print(dResult_final['{}_classification_report'.format(category)])
    print('\n')

toxic
              precision    recall  f1-score   support

           0       0.91      1.00      0.95     57888
           1       0.83      0.03      0.05      6090

   micro avg       0.91      0.91      0.91     63978
   macro avg       0.87      0.51      0.50     63978
weighted avg       0.90      0.91      0.87     63978

              precision    recall  f1-score   support

           0       0.91      1.00      0.95     57888
           1       0.83      0.03      0.05      6090

   micro avg       0.91      0.91      0.91     63978
   macro avg       0.87      0.51      0.50     63978
weighted avg       0.90      0.91      0.87     63978



severe_toxic
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     63611
           1       0.27      0.15      0.19       367

   micro avg       0.99      0.99      0.99     63978
   macro avg       0.63      0.57      0.59     63978
weighted avg       0.99      0.99      0.99     63978